In [8]:
from modules import adquire, cleaning, create_df_checking, outputs
from numpy.random import choice
from time import sleep
import pygame
pygame.mixer.init()

In [9]:
#drumroll = pygame.mixer.Sound('drumroll.wav')
#drumroll.play()

def main(): 
    df_students, df_katas = adquire()    
    df_students, df_katas = cleaning(df_students, df_katas)
    df,hora= create_df_checking(df_students, df_katas)    
    outputs(df,False)
    return df


if __name__ == '__main__':
    df = main()

antoniogarciagiron presentará la kata 80-s-kids-number-2-help-alf-find-his-spaceship


slug,see-you-next-happy-year,80-s-kids-number-2-help-alf-find-his-spaceship
username,,
amorenorp,False,False
alfredozavala14,True,True
anadeluna,False,False
AnaAGG,False,False
angelanavarro,False,False
antoniogarciagiron,True,True
xarlomi,False,False
CristinaCallejo,False,False
Florengutierrez,False,False
